In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import matplotlib.dates as mdates
from bs4 import BeautifulSoup
import requests
import glob
import numpy as np
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os

In [7]:
file_path = "../data/processed/item_price_lag_filled.csv"
data = pd.read_csv(file_path)
data['priceDate'] = pd.to_datetime(data['priceDate'], errors='coerce')
# 2018년 이후 데이터터
data = data[data['priceDate'] >= '2018-01-01']
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 138942 entries, 1041 to 163865
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   priceDate       138942 non-null  datetime64[ns]
 1   item            138942 non-null  object        
 2   market          138942 non-null  object        
 3   minPrice        138942 non-null  float64       
 4   avgPrice        138942 non-null  float64       
 5   maxPrice        138942 non-null  float64       
 6   avgPrice_lag_1  138901 non-null  float64       
 7   avgPrice_lag_2  138860 non-null  float64       
 8   avgPrice_lag_3  138819 non-null  float64       
 9   avgPrice_lag_7  138659 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 11.7+ MB


In [55]:
# 우럭 가격 데이터 로드
price_data = pd.read_csv('../data/processed/item_price_lag_filled.csv')
price_data['priceDate'] = pd.to_datetime(price_data['priceDate'])
price_data = price_data[price_data['priceDate'] >= '2018-01-01']
price_data['날짜'] = price_data['priceDate'].dt.date
rockfish_prices = price_data[price_data['item'] == '우럭'].groupby('날짜')['avgPrice'].mean().reset_index()

# 여러 개의 기상 데이터 파일 불러오기
file_list = glob.glob('../data/raw/forecast_1525/forecast_*.csv')

if not file_list:
    raise FileNotFoundError("기상 데이터 파일이 존재하지 않습니다. 경로를 확인하세요.")

# 각 파일을 개별적으로 처리 후 리스트에 저장
df_list = []
for file in file_list:
    df = pd.read_csv(file, encoding='euc-kr')
    df['일시'] = pd.to_datetime(df['일시'])
    df['날짜'] = df['일시'].dt.date
    df_list.append(df)

# 모든 데이터를 병합
forecast_df = pd.concat(df_list, ignore_index=True)
daily_avg = forecast_df.groupby(['지점', '날짜']).mean().reset_index()

# 지역별 데이터 분리
selected_locations = {'부안': 22186, '가거도': [22297, 22193], '통영': 22188}
df_buan = daily_avg[daily_avg['지점'] == selected_locations['부안']]
df_gageodo = daily_avg[daily_avg['지점'].isin(selected_locations['가거도'])].groupby('날짜').mean().reset_index()
df_tongyoung = daily_avg[daily_avg['지점'] == selected_locations['통영']]

def analyze_correlation(weather_df, price_df, max_lag=30):
    best_lag = 0
    best_corr = -1
    for lag in range(max_lag + 1):
        shifted_prices = price_df.copy()
        shifted_prices['날짜'] = shifted_prices['날짜'] - pd.Timedelta(days=lag)
        merged = pd.merge(shifted_prices, weather_df, on='날짜', how='inner')
        if not merged.empty:
            scaler = StandardScaler()
            
            # 수치형 데이터만 선택
            numeric_cols = merged.select_dtypes(include=['float64', 'int64']).columns
            weather_scaled = scaler.fit_transform(merged[numeric_cols])
            price_scaled = scaler.fit_transform(merged[['avgPrice']])

            corr, _ = pearsonr(price_scaled.flatten(), weather_scaled[:, 0])
            if corr > best_corr:
                best_corr = corr
                best_lag = lag
    return best_lag, best_corr

### 이하 우럭

In [ ]:
# 우럭 가격 데이터 로드
price_data = pd.read_csv('../data/processed/item_price_lag_filled.csv')
price_data['priceDate'] = pd.to_datetime(price_data['priceDate'])
price_data = price_data[price_data['priceDate'] >= '2018-01-01']
price_data['날짜'] = price_data['priceDate'].dt.date
rockfish_prices = price_data[price_data['item'] == '우럭'].groupby('날짜')['avgPrice'].mean().reset_index()

# 여러 개의 기상 데이터 파일 불러오기
file_list = glob.glob('../data/raw/forecast_1525/forecast_*.csv')

df_list = []
for file in file_list:
    df = pd.read_csv(file, encoding='euc-kr')
    df['일시'] = pd.to_datetime(df['일시'])
    df['날짜'] = df['일시'].dt.date
    df_list.append(df)

# 모든 데이터를 병합
forecast_df = pd.concat(df_list, ignore_index=True)
daily_avg = forecast_df.groupby('날짜').mean(numeric_only=True).reset_index()

# 기상 데이터와 가격 데이터 병합
merged_data = pd.merge(rockfish_prices, daily_avg, on='날짜', how='inner')

# 유사도 분석 함수
def calculate_overall_correlation(data):
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
    correlations = {}

    for col in numeric_cols:
        if col != 'avgPrice':
            scaler = StandardScaler()
            weather_scaled = scaler.fit_transform(data[[col]])
            price_scaled = scaler.fit_transform(data[['avgPrice']])
            corr, _ = pearsonr(price_scaled.flatten(), weather_scaled.flatten())
            correlations[col] = corr

    return correlations

# 전체 데이터 상관계수 계산
overall_correlations = calculate_overall_correlation(merged_data)

# 결과 출력
print("전체 지역과 우럭 가격의 상관계수:")
for key, value in overall_correlations.items():
    print(f"{key}: {value:.4f}")

전체 지역과 참돔 가격의 상관계수:
지점: -0.3785
풍속(m/s): 0.2295
풍향(deg): 0.2297
GUST풍속(m/s): 0.2741
현지기압(hPa): 0.3426
습도(%): -0.4572
기온(°C): -0.3452
수온(°C): -0.2187
최대파고(m): 0.2275
유의파고(m): 0.2294
평균파고(m): 0.2306
파주기(sec): 0.2638
파향(deg): 0.1734


### 가거도, 부안, 통영

In [57]:
# 우럭 가격 데이터 로드
price_data = pd.read_csv('../data/processed/item_price_lag_filled.csv')
price_data['priceDate'] = pd.to_datetime(price_data['priceDate'])
price_data = price_data[price_data['priceDate'] >= '2018-01-01']
price_data['날짜'] = price_data['priceDate'].dt.date
rockfish_prices = price_data[price_data['item'] == '우럭'].groupby('날짜')['avgPrice'].mean().reset_index()

# 여러 개의 기상 데이터 파일 불러오기
file_list = glob.glob('../data/raw/forecast_1525/forecast_*.csv')

# 각 파일을 개별적으로 처리 후 리스트에 저장
df_list = []
for file in file_list:
    df = pd.read_csv(file, encoding='euc-kr')
    df['일시'] = pd.to_datetime(df['일시'])
    df['날짜'] = df['일시'].dt.date
    df_list.append(df)

# 모든 데이터를 병합
forecast_df = pd.concat(df_list, ignore_index=True)
daily_avg = forecast_df.groupby(['지점', '날짜']).mean(numeric_only=True).reset_index()

# 지역별 데이터 분리 (부안, 가거도, 통영)
selected_locations = {'부안': 22186, '가거도': [22297, 22193], '통영': 22188}
df_buan = daily_avg[daily_avg['지점'] == selected_locations['부안']]
df_gageodo = daily_avg[daily_avg['지점'].isin(selected_locations['가거도'])].groupby('날짜').mean().reset_index()
df_tongyoung = daily_avg[daily_avg['지점'] == selected_locations['통영']]

# 모든 지역 데이터 통합
all_weather_data = pd.concat([df_buan, df_gageodo, df_tongyoung]).groupby('날짜').mean().reset_index()

# 유사도 분석 함수
def calculate_correlations(weather_df, price_df):
    merged = pd.merge(price_df, weather_df, on='날짜', how='inner')
    numeric_cols = merged.select_dtypes(include=['float64', 'int64']).columns
    correlations = {}

    for col in numeric_cols:
        scaler = StandardScaler()
        weather_scaled = scaler.fit_transform(merged[[col]])
        price_scaled = scaler.fit_transform(merged[['avgPrice']])
        corr, _ = pearsonr(price_scaled.flatten(), weather_scaled.flatten())
        correlations[col] = corr

    return correlations

# 개별 항목과의 상관계수 계산
individual_correlations = calculate_correlations(all_weather_data, rockfish_prices)
individual_correlations

# # # 유의미한 항목 선별 (절대 상관계수 0.3 이상)
# significant_columns = [k for k, v in individual_correlations.items() if abs(v) >= 0.3]

# # 선택된 유의미한 항목만 추출
# if significant_columns:
#     significant_data = all_weather_data[['날짜'] + significant_columns]
#     combined_correlation = calculate_correlations(significant_data, rockfish_prices)
# else:
#     combined_correlation = {}

# # 결과 출력
# individual_correlations, combined_correlation

{'avgPrice': np.float64(1.0),
 '지점': np.float64(-0.2909706006666098),
 '풍속(m/s)': np.float64(0.2746287583498614),
 '풍향(deg)': np.float64(0.2029877026244553),
 'GUST풍속(m/s)': np.float64(0.2872707617250629),
 '현지기압(hPa)': np.float64(0.4037666148048912),
 '습도(%)': np.float64(-0.4341703792340106),
 '기온(°C)': np.float64(-0.3619870290917399),
 '수온(°C)': np.float64(-0.25429761048978494),
 '최대파고(m)': np.float64(0.13499602336488062),
 '유의파고(m)': np.float64(0.14364934288625494),
 '평균파고(m)': np.float64(0.1432662483130067),
 '파주기(sec)': np.float64(0.1851249401592211),
 '파향(deg)': np.float64(0.23659185046220035)}

# 이하 참돔

In [61]:
# 우럭 가격 데이터 로드
price_data = pd.read_csv('../data/processed/item_price_lag_filled.csv')
price_data['priceDate'] = pd.to_datetime(price_data['priceDate'])
price_data = price_data[price_data['priceDate'] >= '2018-01-01']
price_data['날짜'] = price_data['priceDate'].dt.date
rockfish_prices = price_data[price_data['item'] == '참돔'].groupby('날짜')['avgPrice'].mean().reset_index()

# 여러 개의 기상 데이터 파일 불러오기
file_list = glob.glob('../data/raw/forecast_1525/forecast_*.csv')

df_list = []
for file in file_list:
    df = pd.read_csv(file, encoding='euc-kr')
    df['일시'] = pd.to_datetime(df['일시'])
    df['날짜'] = df['일시'].dt.date
    df_list.append(df)

# 모든 데이터를 병합
forecast_df = pd.concat(df_list, ignore_index=True)
daily_avg = forecast_df.groupby('날짜').mean(numeric_only=True).reset_index()

# 기상 데이터와 가격 데이터 병합
merged_data = pd.merge(rockfish_prices, daily_avg, on='날짜', how='inner')

# 유사도 분석 함수
def calculate_overall_correlation(data):
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
    correlations = {}

    for col in numeric_cols:
        if col != 'avgPrice':
            scaler = StandardScaler()
            weather_scaled = scaler.fit_transform(data[[col]])
            price_scaled = scaler.fit_transform(data[['avgPrice']])
            corr, _ = pearsonr(price_scaled.flatten(), weather_scaled.flatten())
            correlations[col] = corr

    return correlations

# 전체 데이터 상관계수 계산
overall_correlations = calculate_overall_correlation(merged_data)

# 결과 출력
print("전체 지역과 참돔 가격의 상관계수:")
for key, value in overall_correlations.items():
    print(f"{key}: {value:.4f}")

전체 지역과 참돔 가격의 상관계수:
지점: 0.7174
풍속(m/s): 0.0461
풍향(deg): 0.0222
GUST풍속(m/s): 0.0614
현지기압(hPa): 0.0759
습도(%): 0.0151
기온(°C): 0.0045
수온(°C): 0.0912
최대파고(m): 0.0114
유의파고(m): 0.0366
평균파고(m): 0.0118
파주기(sec): -0.2234
파향(deg): -0.0484


In [60]:
# 우럭 가격 데이터 로드
price_data = pd.read_csv('../data/processed/item_price_lag_filled.csv')
price_data['priceDate'] = pd.to_datetime(price_data['priceDate'])
price_data = price_data[price_data['priceDate'] >= '2018-01-01']
price_data['날짜'] = price_data['priceDate'].dt.date
rockfish_prices = price_data[price_data['item'] == '참돔'].groupby('날짜')['avgPrice'].mean().reset_index()

# 여러 개의 기상 데이터 파일 불러오기
file_list = glob.glob('../data/raw/forecast_1525/forecast_*.csv')

# 각 파일을 개별적으로 처리 후 리스트에 저장
df_list = []
for file in file_list:
    df = pd.read_csv(file, encoding='euc-kr')
    df['일시'] = pd.to_datetime(df['일시'])
    df['날짜'] = df['일시'].dt.date
    df_list.append(df)

# 모든 데이터를 병합
forecast_df = pd.concat(df_list, ignore_index=True)
daily_avg = forecast_df.groupby(['지점', '날짜']).mean(numeric_only=True).reset_index()

# 지역별 데이터 분리 (부안, 가거도, 통영)
selected_locations = {'가거도': [22297, 22193], '통영': 22188}
df_gageodo = daily_avg[daily_avg['지점'].isin(selected_locations['가거도'])].groupby('날짜').mean().reset_index()
df_tongyoung = daily_avg[daily_avg['지점'] == selected_locations['통영']]

# 모든 지역 데이터 통합
all_weather_data = pd.concat([df_buan, df_gageodo, df_tongyoung]).groupby('날짜').mean().reset_index()

# 유사도 분석 함수
def calculate_correlations(weather_df, price_df):
    merged = pd.merge(price_df, weather_df, on='날짜', how='inner')
    numeric_cols = merged.select_dtypes(include=['float64', 'int64']).columns
    correlations = {}

    for col in numeric_cols:
        scaler = StandardScaler()
        weather_scaled = scaler.fit_transform(merged[[col]])
        price_scaled = scaler.fit_transform(merged[['avgPrice']])
        corr, _ = pearsonr(price_scaled.flatten(), weather_scaled.flatten())
        correlations[col] = corr

    return correlations

# 개별 항목과의 상관계수 계산
individual_correlations = calculate_correlations(all_weather_data, rockfish_prices)
individual_correlations

{'avgPrice': np.float64(1.0),
 '지점': np.float64(0.38527354533824365),
 '풍속(m/s)': np.float64(0.0589457544239286),
 '풍향(deg)': np.float64(-0.008655320766031736),
 'GUST풍속(m/s)': np.float64(0.09594358254995757),
 '현지기압(hPa)': np.float64(0.05675589788194707),
 '습도(%)': np.float64(0.012397045829841281),
 '기온(°C)': np.float64(0.04573803756912134),
 '수온(°C)': np.float64(0.11376503420992212),
 '최대파고(m)': np.float64(0.05878279355867318),
 '유의파고(m)': np.float64(0.08469307854039856),
 '평균파고(m)': np.float64(0.07211038025466467),
 '파주기(sec)': np.float64(-0.27869234991069236),
 '파향(deg)': np.float64(0.027916905034034233)}

# 이하 방어

In [63]:
# 우럭 가격 데이터 로드
price_data = pd.read_csv('../data/processed/item_price_lag_filled.csv')
price_data['priceDate'] = pd.to_datetime(price_data['priceDate'])
price_data = price_data[price_data['priceDate'] >= '2018-01-01']
price_data['날짜'] = price_data['priceDate'].dt.date
rockfish_prices = price_data[price_data['item'] == '방어'].groupby('날짜')['avgPrice'].mean().reset_index()

# 여러 개의 기상 데이터 파일 불러오기
file_list = glob.glob('../data/raw/forecast_1525/forecast_*.csv')

df_list = []
for file in file_list:
    df = pd.read_csv(file, encoding='euc-kr')
    df['일시'] = pd.to_datetime(df['일시'])
    df['날짜'] = df['일시'].dt.date
    df_list.append(df)

# 모든 데이터를 병합
forecast_df = pd.concat(df_list, ignore_index=True)
daily_avg = forecast_df.groupby('날짜').mean(numeric_only=True).reset_index()

# 기상 데이터와 가격 데이터 병합
merged_data = pd.merge(rockfish_prices, daily_avg, on='날짜', how='inner')

# 유사도 분석 함수
def calculate_overall_correlation(data):
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
    correlations = {}

    for col in numeric_cols:
        if col != 'avgPrice':
            scaler = StandardScaler()
            weather_scaled = scaler.fit_transform(data[[col]])
            price_scaled = scaler.fit_transform(data[['avgPrice']])
            corr, _ = pearsonr(price_scaled.flatten(), weather_scaled.flatten())
            correlations[col] = corr

    return correlations

# 전체 데이터 상관계수 계산
overall_correlations = calculate_overall_correlation(merged_data)

# 결과 출력
print("전체 지역과 방어 가격의 상관계수:")
for key, value in overall_correlations.items():
    print(f"{key}: {value:.4f}")

전체 지역과 방어 가격의 상관계수:
지점: 0.6250
풍속(m/s): 0.0516
풍향(deg): 0.0705
GUST풍속(m/s): 0.0643
현지기압(hPa): 0.0428
습도(%): 0.0240
기온(°C): -0.0695
수온(°C): -0.0147
최대파고(m): 0.0418
유의파고(m): 0.0609
평균파고(m): 0.0395
파주기(sec): -0.2475
파향(deg): -0.0053


In [76]:
# 우럭 가격 데이터 로드
price_data = pd.read_csv('../data/processed/item_price_lag_filled.csv')
price_data['priceDate'] = pd.to_datetime(price_data['priceDate'])
price_data = price_data[price_data['priceDate'] >= '2018-01-01']
price_data['날짜'] = price_data['priceDate'].dt.date
rockfish_prices = price_data[price_data['item'] == '방어'].groupby('날짜')['avgPrice'].mean().reset_index()

# 여러 개의 기상 데이터 파일 불러오기
file_list = glob.glob('../data/raw/forecast_1525/forecast_*.csv')

# 각 파일을 개별적으로 처리 후 리스트에 저장
df_list = []
for file in file_list:
    df = pd.read_csv(file, encoding='euc-kr')
    df['일시'] = pd.to_datetime(df['일시'])
    df['날짜'] = df['일시'].dt.date
    df_list.append(df)

# 모든 데이터를 병합
forecast_df = pd.concat(df_list, ignore_index=True)
daily_avg = forecast_df.groupby(['지점', '날짜']).mean(numeric_only=True).reset_index()

# 지역별 데이터 분리 (부안, 가거도, 통영)
selected_locations = {'가거도': [22297, 22193], '통영': 22188}
df_gageodo = daily_avg[daily_avg['지점'].isin(selected_locations['가거도'])].groupby('날짜').mean().reset_index()
df_tongyoung = daily_avg[daily_avg['지점'] == selected_locations['통영']]

# 모든 지역 데이터 통합
all_weather_data = pd.concat([df_buan, df_gageodo, df_tongyoung]).groupby('날짜').mean().reset_index()

# 유사도 분석 함수
def calculate_correlations(weather_df, price_df):
    merged = pd.merge(price_df, weather_df, on='날짜', how='inner')
    numeric_cols = merged.select_dtypes(include=['float64', 'int64']).columns
    correlations = {}

    for col in numeric_cols:
        scaler = StandardScaler()
        weather_scaled = scaler.fit_transform(merged[[col]])
        price_scaled = scaler.fit_transform(merged[['avgPrice']])
        corr, _ = pearsonr(price_scaled.flatten(), weather_scaled.flatten())
        correlations[col] = corr

    return correlations

# 개별 항목과의 상관계수 계산
individual_correlations = calculate_correlations(all_weather_data, rockfish_prices)
individual_correlations

{'avgPrice': np.float64(0.9999999999999998),
 '지점': np.float64(0.1196726431971852),
 '풍속(m/s)': np.float64(0.054406924371199566),
 '풍향(deg)': np.float64(0.07521079533801875),
 'GUST풍속(m/s)': np.float64(0.08824183040868319),
 '현지기압(hPa)': np.float64(0.031169153502364537),
 '습도(%)': np.float64(0.02541044103838705),
 '기온(°C)': np.float64(-0.010418222121085813),
 '수온(°C)': np.float64(0.030373423953473443),
 '최대파고(m)': np.float64(0.03640739668664986),
 '유의파고(m)': np.float64(0.05425884643807624),
 '평균파고(m)': np.float64(0.043455911982901024),
 '파주기(sec)': np.float64(-0.23389196543514995),
 '파향(deg)': np.float64(0.07463536491862163)}

# 대게

In [66]:
# 우럭 가격 데이터 로드
price_data = pd.read_csv('../data/processed/item_price_lag_filled.csv')
price_data['priceDate'] = pd.to_datetime(price_data['priceDate'])
price_data = price_data[price_data['priceDate'] >= '2018-01-01']
price_data['날짜'] = price_data['priceDate'].dt.date
rockfish_prices = price_data[price_data['item'] == '대게'].groupby('날짜')['avgPrice'].mean().reset_index()

# 여러 개의 기상 데이터 파일 불러오기
file_list = glob.glob('../data/raw/forecast_1525/forecast_*.csv')

df_list = []
for file in file_list:
    df = pd.read_csv(file, encoding='euc-kr')
    df['일시'] = pd.to_datetime(df['일시'])
    df['날짜'] = df['일시'].dt.date
    df_list.append(df)

# 모든 데이터를 병합
forecast_df = pd.concat(df_list, ignore_index=True)
daily_avg = forecast_df.groupby('날짜').mean(numeric_only=True).reset_index()

# 기상 데이터와 가격 데이터 병합
merged_data = pd.merge(rockfish_prices, daily_avg, on='날짜', how='inner')

# 유사도 분석 함수
def calculate_overall_correlation(data):
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
    correlations = {}

    for col in numeric_cols:
        if col != 'avgPrice':
            scaler = StandardScaler()
            weather_scaled = scaler.fit_transform(data[[col]])
            price_scaled = scaler.fit_transform(data[['avgPrice']])
            corr, _ = pearsonr(price_scaled.flatten(), weather_scaled.flatten())
            correlations[col] = corr

    return correlations

# 전체 데이터 상관계수 계산
overall_correlations = calculate_overall_correlation(merged_data)

# 결과 출력
print("전체 지역과 대게 가격의 상관계수:")
for key, value in overall_correlations.items():
    print(f"{key}: {value:.4f}")

전체 지역과 대게게 가격의 상관계수:
지점: 0.3759
풍속(m/s): 0.2975
풍향(deg): 0.2053
GUST풍속(m/s): 0.3443
현지기압(hPa): 0.5041
습도(%): -0.4940
기온(°C): -0.4931
수온(°C): -0.3466
최대파고(m): 0.2829
유의파고(m): 0.2914
평균파고(m): 0.2795
파주기(sec): 0.1540
파향(deg): 0.0635


In [78]:
# 대게 가격 데이터 로드
price_data = pd.read_csv('../data/processed/item_price_lag_filled.csv')
price_data['priceDate'] = pd.to_datetime(price_data['priceDate'])
price_data = price_data[price_data['priceDate'] >= '2018-01-01']
price_data['날짜'] = price_data['priceDate'].dt.date
rockfish_prices = price_data[price_data['item'] == '대게'].groupby('날짜')['avgPrice'].mean().reset_index()

# 여러 개의 기상 데이터 파일 불러오기
file_list = glob.glob('../data/raw/forecast_1525/forecast_*.csv')

# 각 파일을 개별적으로 처리 후 리스트에 저장
df_list = []
for file in file_list:
    df = pd.read_csv(file, encoding='euc-kr')
    df['일시'] = pd.to_datetime(df['일시'])
    df['날짜'] = df['일시'].dt.date
    df_list.append(df)

# 모든 데이터를 병합
forecast_df = pd.concat(df_list, ignore_index=True)
daily_avg = forecast_df.groupby(['지점', '날짜']).mean(numeric_only=True).reset_index()

# 지역별 데이터 분리 (부안, 가거도, 통영)
selected_locations = {'부안': [22186]}
df_gageodo = daily_avg[daily_avg['지점'].isin(selected_locations['부안'])].groupby('날짜').mean().reset_index()

# 모든 지역 데이터 통합
all_weather_data = pd.concat([df_buan, df_gageodo, df_tongyoung]).groupby('날짜').mean().reset_index()

# 유사도 분석 함수
def calculate_correlations(weather_df, price_df):
    merged = pd.merge(price_df, weather_df, on='날짜', how='inner')
    numeric_cols = merged.select_dtypes(include=['float64', 'int64']).columns
    correlations = {}

    for col in numeric_cols:
        scaler = StandardScaler()
        weather_scaled = scaler.fit_transform(merged[[col]])
        price_scaled = scaler.fit_transform(merged[['avgPrice']])
        corr, _ = pearsonr(price_scaled.flatten(), weather_scaled.flatten())
        correlations[col] = corr

    return correlations

# 개별 항목과의 상관계수 계산
individual_correlations = calculate_correlations(all_weather_data, rockfish_prices)
individual_correlations

{'avgPrice': np.float64(0.9999999999999998),
 '지점': np.float64(-0.062196129817206985),
 '풍속(m/s)': np.float64(0.30054166259938275),
 '풍향(deg)': np.float64(0.10262992590325193),
 'GUST풍속(m/s)': np.float64(0.3285639726206786),
 '현지기압(hPa)': np.float64(0.521052311015256),
 '습도(%)': np.float64(nan),
 '기온(°C)': np.float64(-0.5116191346520339),
 '수온(°C)': np.float64(-0.3867970906283041),
 '최대파고(m)': np.float64(0.25894650729983065),
 '유의파고(m)': np.float64(0.2670293735014785),
 '평균파고(m)': np.float64(0.2646177465494437),
 '파주기(sec)': np.float64(0.018008969607395835),
 '파향(deg)': np.float64(0.20783479976598376)}

#   이하 광어

In [75]:
# 우럭 가격 데이터 로드
price_data = pd.read_csv('../data/processed/item_price_lag_filled.csv')
price_data['priceDate'] = pd.to_datetime(price_data['priceDate'])
price_data = price_data[price_data['priceDate'] >= '2018-01-01']
price_data['날짜'] = price_data['priceDate'].dt.date
rockfish_prices = price_data[price_data['item'] == '광어'].groupby('날짜')['avgPrice'].mean().reset_index()

# 여러 개의 기상 데이터 파일 불러오기
file_list = glob.glob('../data/raw/forecast_1525/forecast_*.csv')

df_list = []
for file in file_list:
    df = pd.read_csv(file, encoding='euc-kr')
    df['일시'] = pd.to_datetime(df['일시'])
    df['날짜'] = df['일시'].dt.date
    df_list.append(df)

# 모든 데이터를 병합
forecast_df = pd.concat(df_list, ignore_index=True)
daily_avg = forecast_df.groupby('날짜').mean(numeric_only=True).reset_index()

# 기상 데이터와 가격 데이터 병합
merged_data = pd.merge(rockfish_prices, daily_avg, on='날짜', how='inner')

# 유사도 분석 함수
def calculate_overall_correlation(data):
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
    correlations = {}

    for col in numeric_cols:
        if col != 'avgPrice':
            scaler = StandardScaler()
            weather_scaled = scaler.fit_transform(data[[col]])
            price_scaled = scaler.fit_transform(data[['avgPrice']])
            corr, _ = pearsonr(price_scaled.flatten(), weather_scaled.flatten())
            correlations[col] = corr

    return correlations

# 전체 데이터 상관계수 계산
overall_correlations = calculate_overall_correlation(merged_data)

# 결과 출력
print("전체 지역과 광어 가격의 상관계수:")
for key, value in overall_correlations.items():
    print(f"{key}: {value:.4f}")

전체 지역과 광어 가격의 상관계수:
지점: 0.7861
풍속(m/s): 0.0574
풍향(deg): 0.0240
GUST풍속(m/s): 0.0753
현지기압(hPa): 0.0801
습도(%): 0.0002
기온(°C): -0.0179
수온(°C): 0.0558
최대파고(m): 0.0213
유의파고(m): 0.0469
평균파고(m): 0.0241
파주기(sec): -0.1448
파향(deg): -0.0109


In [74]:
# 우럭 가격 데이터 로드
price_data = pd.read_csv('../data/processed/item_price_lag_filled.csv')
price_data['priceDate'] = pd.to_datetime(price_data['priceDate'])
price_data = price_data[price_data['priceDate'] >= '2018-01-01']
price_data['날짜'] = price_data['priceDate'].dt.date
rockfish_prices = price_data[price_data['item'] == '광어'].groupby('날짜')['avgPrice'].mean().reset_index()

# 여러 개의 기상 데이터 파일 불러오기
file_list = glob.glob('../data/raw/forecast_1525/forecast_*.csv')

# 각 파일을 개별적으로 처리 후 리스트에 저장
df_list = []
for file in file_list:
    df = pd.read_csv(file, encoding='euc-kr')
    df['일시'] = pd.to_datetime(df['일시'])
    df['날짜'] = df['일시'].dt.date
    df_list.append(df)

# 모든 데이터를 병합
forecast_df = pd.concat(df_list, ignore_index=True)
daily_avg = forecast_df.groupby(['지점', '날짜']).mean(numeric_only=True).reset_index()

# 지역별 데이터 분리 (부안, 가거도, 통영)
selected_locations = {'가거도': [22297, 22193], '마라도': 22188, '통영': 22188}
df_gageodo = daily_avg[daily_avg['지점'].isin(selected_locations['가거도'])].groupby('날짜').mean().reset_index()
df_tongyoung = daily_avg[daily_avg['지점'] == selected_locations['통영']]
df_marado = daily_avg[daily_avg['지점'] == selected_locations['마라도']]

# 모든 지역 데이터 통합
all_weather_data = pd.concat([df_buan, df_gageodo, df_tongyoung]).groupby('날짜').mean().reset_index()

# 유사도 분석 함수
def calculate_correlations(weather_df, price_df):
    merged = pd.merge(price_df, weather_df, on='날짜', how='inner')
    numeric_cols = merged.select_dtypes(include=['float64', 'int64']).columns
    correlations = {}

    for col in numeric_cols:
        scaler = StandardScaler()
        weather_scaled = scaler.fit_transform(merged[[col]])
        price_scaled = scaler.fit_transform(merged[['avgPrice']])
        corr, _ = pearsonr(price_scaled.flatten(), weather_scaled.flatten())
        correlations[col] = corr

    return correlations

# 개별 항목과의 상관계수 계산
individual_correlations = calculate_correlations(all_weather_data, rockfish_prices)
individual_correlations

{'avgPrice': np.float64(1.0),
 '지점': np.float64(0.5372792955026775),
 '풍속(m/s)': np.float64(0.06442489586061431),
 '풍향(deg)': np.float64(-0.006991582286604785),
 'GUST풍속(m/s)': np.float64(0.10348563396205601),
 '현지기압(hPa)': np.float64(0.07840365716056726),
 '습도(%)': np.float64(-0.013587590042486906),
 '기온(°C)': np.float64(0.005764803272428218),
 '수온(°C)': np.float64(0.07661706420517839),
 '최대파고(m)': np.float64(0.07568211235681843),
 '유의파고(m)': np.float64(0.10095978866152949),
 '평균파고(m)': np.float64(0.09050118317151007),
 '파주기(sec)': np.float64(-0.19212964624737272),
 '파향(deg)': np.float64(0.04337381929813404)}